In [1]:
import csv
import os
import itertools as itt
import classes as vp
import plotly.graph_objects as go
import plot
import mathHelper as mh

In [2]:
def loadSessions(file = 'sessions.csv', csv_data_dir = './csv_data'):
    #load session data form sessions.csv
    sessions = [] #list of all the sessions
    with open(file) as sessions_csv:
        sessionno = 0
        csv_reader = csv.reader(sessions_csv, delimiter=',')
        for l in csv_reader:
            sessions.append(vp.Session(sessionno, l[1], l[2], csv_data_dir + '/' + l[3]))
            sessionno += 1
    return sessions   

In [3]:
#todo: need output
def loadFactions(file = 'factions.csv', period='50'):
    factions_count = dict()
    with open(file) as factions_csv:
        csv_reader = csv.reader(factions_csv, delimiter=',')
        l = next(csv_reader)
        periodIndex = None
        for i,item in enumerate(l):
            if item == period:
                periodIndex = i
                break
        if periodIndex == None:
            #could not fined period
            return None
            
        l = next(csv_reader)
        while True:
            try:
                factions_count[l[0]] = int(l[periodIndex])
                l = next(csv_reader)
            except StopIteration:
                break
        return factions_count

In [4]:
def parseMems(r, session):
    #members = dict()
    posIdMap = []
    line = next(r)
    start = 0
    end = 0
    for i, item in enumerate(line):
        if item == 'CouncillorId':
            start = i+1
            break
    
    line = next(r)
    
    #enumerate ID
    for i, item in enumerate(line[start:]):
        if item == '':
            end = i + start
            break
            
        posIdMap.append(item)
        if not item in members:
            members[item] = vp.Member(item)
        (members[item].ismem).append(session.idno)

    ''' 
    #enumerate bioid
    line = next(r)    
    for i,item in enumerate(line[start:end]):
        members[posIdMap[i]].bioid = item
    '''
    
    #enumerate name
    line = next(r)
    for i,item in enumerate(line[start:end]):
        #print(i), print(item)
        members[posIdMap[i]].name = item
        
    #skip line (this line only shows members are part of national council)
    next(r)
    #enumerate faction membership
    line = next(r)
    for i,item in enumerate(line[start:end]):
        members[posIdMap[i]].faction = item 
        
    #enumerate canton
    line = next(r)
    for i,item in enumerate(line[start:end]):
        members[posIdMap[i]].canton = item
        
    #enumerate birthdata
    line = next(r)
    for i,item in enumerate(line[start:end]):
        members[posIdMap[i]].birthdate = item

    

In [5]:
#parses all the sessions for the votes
def parseVotes(r, s):
    l = next(r)
    voters = []
    #votes = []
    voteno = 0
    start = 0
    while l[0] != 'VoteDate':
        l = next(r)
        start += 1
    
    #voters contains all the voters in a session
    endVotes = 0
    for i, item in enumerate(l[12:]):
        if item != 'Decision':
            voters.append(item)
        else:
            endVotes = i + 12
            break
    #print(len(voters))
    
    g = set()
    l = next(r)
    
    
    #need to handle:
    vote_none = '' #member was not yet or is no longer a national coucil member
    vote_yes = 'Ja'
    vote_no = 'Nein'
    vote_novote = ['Entschuldigt', 'Hat nicht teilgenommen']
    vote_abstain = 'Enthaltung'
    vote_pres = 'Der Präsident stimmt nicht'
    decision_set = {'yes', 'no'}
    
    
    while True:
        try:
            #store vote meta data
            v = vp.Vote(voteno)
            v.date = l[0]
            v.affairId = l[4]
            v.title = l[5]
            v.session = s
            v.sessionId = s.idno
            #store how each member voted by adding the members bioid to the yes/no/abstain/noset set
            for i, item in enumerate(l[12:endVotes]):
                if item == vote_none:
                    pass
                elif item == vote_yes:
                    v.yes.add(voters[i])
                elif item == vote_no:
                    v.no.add(voters[i])
                elif item == vote_abstain:
                    v.abstain.add(voters[i])
                elif item in vote_novote:
                    v.novote.add(voters[i])
                elif item == vote_pres:
                    pass
                else:
                    print('err vote: ' + item)
            
            #store decision data as stored in file and compare with calculated decision value
            v.decision = l[endVotes].lower()
            
            if v.decision != v.getDecision() or (not v.decision in decision_set):
                v.decision = 'err: \"' + v.decision + '\"' 
            votes.append(v)
            voteno += 1
            l = next(r)
        except StopIteration:
            break

In [42]:
#simple impirical voting power
#count how often a party in pivotal
#uses votes
def impVotingPower1(memList):
    pivotal = dict()
    for f in memList:
        count = 0
        for v in votes:
            if v.isPivotal(memList[f]):
                count += 1
        pivotal[vp.faction_names[f]] = count / len(votes)
    return pivotal

In [7]:
# find the wining coelition for all the votes
# needs member dict
def allWinningC(votes, ids=False):
    coelitions = dict()
    
    for i in range(1, len(vp.party_names) + 1):
        for p in itt.combinations(vp.party_names, i):
            key = list(p)
            key.sort(key=lambda x: vp.party_order[x])
            if ids:
                coelitions[', '.join(key)] = []
            else:
                coelitions[','.join(key)] = 0
    
    for i, v in enumerate(votes):
        try:
            #change to lists vot ids instead
            if ids:
                coelitions[', '.join(winningC(v))].append(i)
            else:
                coelitions[','.join(winningC(v))] += 1
        except KeyError:
            print(v)
            print(i)
        
    return coelitions
    
    
#returns winning coelition for a vote
def winningC(vote):
    profiles = dict()
    for f in vp.party_names:
        profiles[f] = vp.VoteProfile()
    
    for mem in vote.yes:
        profiles[vp.faction_names[members[mem].faction]].yes += 1
        
    for mem in vote.no:
        profiles[vp.faction_names[members[mem].faction]].no += 1
        
    for mem in vote.abstain:
        profiles[vp.faction_names[members[mem].faction]].abstain += 1
        
    for mem in vote.novote:
        profiles[vp.faction_names[members[mem].faction]].novote += 1
        
    out = list()
    
    for p in profiles:
        if profiles[p].partyVote(abstain=False) == vote.decision:
            out.append(p)
    
    out.sort(key=lambda x: vp.party_order[x])
    return out
        
    

In [8]:
def profie(vote):
    profiles = dict()
    for f in vp.party_names:
        profiles[f] = vp.VoteProfile()
    
    for mem in vote.yes:
        profiles[vp.faction_names[members[mem].faction]].yes += 1
        
    for mem in vote.no:
        profiles[vp.faction_names[members[mem].faction]].no += 1
        
    for mem in vote.abstain:
        profiles[vp.faction_names[members[mem].faction]].abstain += 1
        
    for mem in vote.novote:
        profiles[vp.faction_names[members[mem].faction]].novote += 1
        
    for p in profiles:
        print(p + ' ' + str(profiles[p]))

In [9]:
def parseFiles(sessions, toParse = {}): 
    #parse all the files
    #for testing: "filesToParse" can limit the parsed files (everything if empty)
    for s in sessions:
        if s.idno in toParse or len(toParse) == 0:
            with open(s.path) as csv_file:
                csv_reader = csv.reader(csv_file, delimiter=',')
                print('-', end='')
                #print(str(key) + '. reading ... ' + csv_file.name)
                #parseFile(csv_reader)
                members = parseMems(csv_reader, s)
                parseVotes(csv_reader, s)
                csv_file.close

In [10]:
def genFactions():
    #list of members for each faction
    factions = dict()
    for mem in members:
        f = members[mem].faction
        if f in factions:
            factions[f].append(members[mem].bioid)
        else:
            factions[f] = [members[mem].bioid]
    return factions

In [50]:
def genFactionSize():
    #dict with size of each faction
    factions_size = dict()
    #factions_size['total'] = 0
    for f in factions:
        factions_size[f] = len(factions[f])
        #factions_size['total'] += len(factions[f])
        
    return factions_size

In [51]:
members = dict()
votes = []
memno = []
leg_49 = set(range(20))
leg_50 = set(range(20,39))

In [52]:
sessions = loadSessions()
parseFiles(sessions, toParse=leg_50)
factions_count_49 = loadFactions(period='49')
factions_count_50 = loadFactions(period='50')
factions = genFactions()
factions_size = genFactionSize()

-------------------

In [14]:
allWinningC(votes)

{'CVP': 0,
 'SVP': 2,
 'GSP': 0,
 'BDP': 0,
 'SP': 0,
 'None': 0,
 'GLP': 0,
 'FDP': 0,
 'CVP,SVP': 38,
 'GSP,CVP': 0,
 'CVP,BDP': 0,
 'SP,CVP': 0,
 'CVP,None': 0,
 'GLP,CVP': 0,
 'CVP,FDP': 0,
 'GSP,SVP': 0,
 'BDP,SVP': 1,
 'SP,SVP': 1,
 'SVP,None': 0,
 'GLP,SVP': 1,
 'FDP,SVP': 150,
 'GSP,BDP': 0,
 'SP,GSP': 0,
 'GSP,None': 0,
 'GSP,GLP': 0,
 'GSP,FDP': 0,
 'SP,BDP': 0,
 'BDP,None': 0,
 'GLP,BDP': 0,
 'BDP,FDP': 0,
 'SP,None': 0,
 'SP,GLP': 0,
 'SP,FDP': 0,
 'GLP,None': 0,
 'FDP,None': 0,
 'GLP,FDP': 0,
 'GSP,CVP,SVP': 8,
 'CVP,BDP,SVP': 64,
 'SP,CVP,SVP': 4,
 'CVP,SVP,None': 0,
 'GLP,CVP,SVP': 0,
 'CVP,FDP,SVP': 189,
 'GSP,CVP,BDP': 0,
 'SP,GSP,CVP': 7,
 'GSP,CVP,None': 0,
 'GSP,GLP,CVP': 0,
 'GSP,CVP,FDP': 0,
 'SP,CVP,BDP': 1,
 'CVP,BDP,None': 0,
 'GLP,CVP,BDP': 0,
 'CVP,BDP,FDP': 0,
 'SP,CVP,None': 0,
 'SP,GLP,CVP': 0,
 'SP,CVP,FDP': 1,
 'GLP,CVP,None': 0,
 'CVP,FDP,None': 0,
 'GLP,CVP,FDP': 0,
 'GSP,BDP,SVP': 0,
 'SP,GSP,SVP': 31,
 'GSP,SVP,None': 0,
 'GSP,GLP,SVP': 1,
 'GSP,FDP,

In [15]:
profie(votes[1057])

CVP party vote: yes yes: 28 no:0 abstain: 0 novote: 2 total: 30
SVP party vote: no yes: 0 no:68 abstain: 0 novote: 0 total: 68
GSP party vote: abstain yes: 0 no:0 abstain: 12 novote: 0 total: 12
BDP party vote: yes yes: 7 no:0 abstain: 0 novote: 0 total: 7
SP party vote: abstain yes: 0 no:0 abstain: 43 novote: 0 total: 43
None party vote: draw yes: 0 no:0 abstain: 0 novote: 0 total: 0
GLP party vote: yes yes: 7 no:0 abstain: 0 novote: 0 total: 7
FDP party vote: yes yes: 25 no:6 abstain: 0 novote: 1 total: 32


In [16]:
print(votes[1057])

24# 267 no yes:67 no:74 abstain:55 novote:3


In [29]:
#displays a histogram of all winning coelitions
def coelitions(simpleName = False, normalize=False, cut=0, ele=0):
    l = list(filter(lambda x: x[1] > cut, allWinningC(votes).items()))
    l.sort(key=lambda x: x[1], reverse=True)
    
    x = []
    y = []
    
    for i in l:
        if simpleName and (i[0] in coelition_names):
            x.append(coelition_names[i[0]])
        else:
            x.append(i[0])
        y.append(i[1])
    
    if normalize:
        #normalize data
        mh.norm(y, percet=True)
    
    print(sum(y))
    
    if ele != 0:
        x = x[:ele]
        y = y[:ele]
        
    #print(y[:40])
    print(sum(y[:20]))
    
    
    fig = go.Figure([go.Bar(x=x, y=y)])
    fig.show()

#simpler names for coelitions
coelition_names = {'SP,GSP,GLP,CVP,BDP,FDP': 'against SVP',
                  'GLP,CVP,BDP,FDP,SVP': 'against left',
                  'SP,GSP,GLP,CVP,BDP,FDP,SVP': 'unanimous',
                  'CVP,BDP,FDP,SVP': 'center right',
                  'SP,GSP,GLP,CVP,BDP': 'center left',
                  'CVP, FDP, SVP': 'right and CVP',
                  'BDP, FDP, SVP': 'right and BDP',
                  'SP, GSP, GLP, BDP, FDP': 'center left'}
coelitions(normalize=True, cut=0, ele=40, simpleName=False)

100.0000000000002
91.6703985669503


In [18]:
def showVoteDist(votes):
    #histogramm of how many members votes yes, no, abstain per vote
    present = []
    absent = []
    for v in votes:
        s = len(v.yes) + len(v.no) + len(v.abstain)
        present.append(s)
        absent.append(len(v.novote))
    print('present memers')
    plot.hist(present)
    print('absent members')
    plot.hist(absent)
showVoteDist(votes)

present memers


absent members


In [19]:
# creates bar chart with member partisipation rates per faction in votes
def memberPartisipation(votes):
    partisipation = dict()
    memPart = vp.KeyList()
    
    def update(key, i, d=partisipation):
        
        key = vp.faction_names[members[key].faction]
        
        if not key in d:
            d[key] = [0,0,0,0]
        else:
            d[key][i] += 1
    
    def presents(x):
        return sum(x[0:3]) / sum(x)
    
    
    for v in votes:
        for mem in v.yes:
            update(mem, 0)
        for mem in v.no:
            update(mem, 1)
        for mem in v.abstain:
            update(mem, 2)
        for mem in v.novote:
            update(mem, 3)  
    
    
    
    plot.bar_dict(partisipation, value=lambda x: presents(x) * 100, relativ= True, sort= True)
memberPartisipation(votes)

In [20]:
# creates bar chart for party unity in votes
def factionUnity(votes, factions):
    factionUnity = dict()
    for f in factions:
        factionUnity[vp.faction_names[f]] = []
        
    for v in votes:
        for f in factions:
            factionUnity[vp.faction_names[f]].append(v.unity(factions[f], abstain=True))
    plot.bar_dict(factionUnity, value= lambda x: sum(x) / len(x) * 100, relativ=False, sort=True)

In [21]:
factionUnity(votes, factions)

In [53]:
def groupedBar_Dict(dicts, names, log=False):
    data = []
    
    for k, d in enumerate(dicts):
        l = []
        for i in d:
            l.append(d[i] * 100)
        
        #for i,item in enumerate(l):
         #   l[i] /= sum(l)
        #norm(l)
        data.append(go.Bar(name=names[k], x=list(d.keys()), y=l))#, marker_color=plot.faction_color))

    fig = go.Figure(data=data)
    # Change the bar mode
    if log:
        fig.update_layout(barmode='group', yaxis_type='log')
    else:
        fig.update_layout(barmode='group')
    fig.show()
    
groupedBar_Dict([impVotingPower1(factions), vp.PBI_votingPower(factions_count_50)], ['imp','50'])

In [23]:
m = []
m.append(['a', 3])
m.append(['b',5])

In [49]:
factions_size

{'SVP': 73, 'SP': 52, 'CVP': 36, 'GSP': 15, 'FDP': 36, 'GLP': 8, 'BDP': 8}

In [41]:
vp.faction_names

{'V': 'SVP',
 'S': 'SP',
 'RL': 'FDP',
 'C': 'CVP',
 'CE': 'CVP',
 'G': 'GSP',
 'BD': 'BDP',
 'GL': 'GLP',
 '-': 'None'}

In [36]:
vp.PBI_votingPower(factions_count_50)

{'SVP': 0.71875,
 'SP': 0.28125,
 'FDP': 0.28125,
 'CVP': 0.21875,
 'GSP': 0.03125,
 'BDP': 0.03125,
 'GLP': 0.03125}

In [25]:
a = [1,2]
b = ['b', 'a']

a = list(map(lambda x: x *100, a))
print(a)

[100, 200]
